# Hands-On Feature Engineering
***

**Reminder**:  Go to the botttom of the notebook and shift-enter the [helper functions](#helpers)
***



The success of a machine learning algorithm on a predictive problem is highly dependent on the way you as the practitioner present the data. Consider the following example. 

### Problem 1: Intuition
***
Suppose that you want to train a model to predict whether it is possible to drive between two cities in a single day.  The raw data includes the latitude and longitude of the two cities and the training data is labeled with $\texttt{Yes}$ for is drivable and $\texttt{No}$ for is not drivable.  One particular training set my look like 

|$\texttt{CITY 1 LAT.}$ | $\texttt{CITY 1 LNG.}$ |$\texttt{CITY 2 LAT.}$ | $\texttt{CITY 2 LNG.}$ | $\texttt{DRIVABLE}$? | 
|:----:|:----:|:----:|:----:|:----:|
| 123.24 |46.71	| 121.33| 47.34	| Yes |
|123.24	|56.91	|121.33	|55.23	|Yes |
|123.24	|46.71	|121.33	|55.34	|No |
|123.24	|46.71	|130.99	|47.34	|No |

**Q**: Given the following features, do you expect a linear classifier like Logistic Regression to be successful? 

**Q**: What features could you create that *would* be correlated with the correct classification? 

**Q**: Could we come up with a process to do this automatically?  Next question:  should we come up with a process to do this automatically? 



<br>

### Problem 2: Transformations on Continuous Data 
***

Consider the case when you're trying to model quality of life of a person from survey data which asks a multitude of things like $\texttt{income}$, $\texttt{education level}$, $\texttt{num children}$, etc.  As part of data exploration, you might try plotting the distributions of the raw data by feature individually.  A histogram of a potential data set for income might look as follows:  


In [ ]:
x_inc = income_data()
fig, ax = plt.subplots(nrows=1, ncols=1, figsize=(12,6))
n, bins, patches = ax.hist(x_inc, 15, normed=1, facecolor=mycolors["red"], alpha=0.75, edgecolor="white")
ax.set_xlabel("income in tens of thousands", fontsize=16)
ax.set_ylabel("frequency", fontsize=16);
ax.set_title("Income Frequency", fontsize=20);
ax.grid(alpha=0.25)

This looks like we would expect for income.  There are lots of people at the low end of the spectrum, a few more in the middle, and very few in the high income categories.  

There are two things to consider here.  First, lots of ML models assume that your input features are generally normally distributed.  Second, from a feature engineering perspective, we can think about how much income actually affects happiness.  It seems reasonable to believe that once you get to a certain level, increasing income has a diminishing effect on happiness.  

Both of these viewpoints motivate us to try a log transformation on the data. 

In [ ]:
x_inc_log = np.log(x_inc)
fig, ax = plt.subplots(nrows=1, ncols=1, figsize=(12,6))
n, bins, patches = ax.hist(x_inc_log, 15, normed=1, facecolor=mycolors["blue"], alpha=0.75, edgecolor="white")
ax.set_xlabel("log(income in thousands)", fontsize=16)
ax.set_ylabel("frequency", fontsize=16);
ax.set_title("Income Frequency", fontsize=20);
ax.grid(alpha=0.25)

This results in a distribution that looks fairly Gaussian, which might perform better in an machine learning model. 

<br>

### Problem 3: Categorical Data 
*** 

Encoding categorical data can be tricky.  Consider the case when you have a raw feature in your model that corresponds to a person's hair color.  Possible values might be $\texttt{blonde}$, $\texttt{brunette}$, $\texttt{redhead}$.  How should we encode these as numerical models in a ML algorithm? 

A natural (but misleading) thing to do would be to assign an integer to each possible value of the feature.  For instance, we could do 

|Hair Color| Feature|
|:-------:|:------:|
|blonde| 0 |
|brunette | 1 | 
|redhead | 2|

**Q**:  What is potentially wrong with this encoding, particularly in a regression context? 

One way to fix this is create binary features corresponding to each religion.  That is, we create a binary $\texttt{IsBlonde}$ variable, a binary $\texttt{IsBrunette}$ variable, etc.  We then have 

|Religion| $\texttt{IsBlonde}$ | $\texttt{IsBrunette}$ | $\texttt{IsRedhead}$| 
|:-------:|:------:|:------:|:------:|
|Blonde| 1 | 0 | 0 | 
|Brunette | 0 | 1 | 0 | 
| Redhead | 0| 0 | 1 

This process is called *one-hot-encoding* and is very frequently used to encode categorical data. 


<br><br>


### Problem 4: Temporal Data 
*** 

Suppose that you're trying to train a model to predict that amount of foot-traffic at the 29th Street Mall.  Mall managers might be interested in such a model to predict the amount of janitorial and security services they need to employ at different times.  

Suppose your training data consists of measurements of the amount of foot traffic at the mall and the date/time stamps that they were measured.  For instance, a training set might look like 

|$\texttt{date_time_stamp}$| $\texttt{FootTraffic}$|
|:-------:|:------:|
|$\texttt{2015-11-12-20:00}$| 70|
|$\texttt{2015-06-10-21:00}$| 100|
|$\texttt{2015-08-02-12:00}$| 120|
|$\texttt{2015-12-22-12:00}$| 20|


**Q**: How might you create meaningful features on the $\texttt{date_time_stamp}$ data that would be useful for prediction? 




<br>
<br>

### Problem 5: Part-of-Speech-Tagging
*This problem and associated code was adapted from Jordan Boyd-Graber*
***

In computational linguistics, part-of-speech tagging (POST) is the process of marking a word in a text as a particular part of speech (e.g. noun, verb, adjective, etc), based on both its definition and its context. 

In this problem we will work with the <a href="https://en.wikipedia.org/wiki/Brown_Corpus">Brown Corpus</a>, a compilation of 500 samples of English-language text totaling over a million words. The Brown Corpus is available through python's <a href="http://www.nltk.org/">Natural Language Toolkit</a>.  Each word in the corpus has been tagged as: 

|type|symbol|
|:--:|:----:|
|adjective| JJ|
|noun|NN|
|pronoun|PP|
|adverb|RB|
|verb|VB|

For the classification we will use simple Logistic Regression and focus on making iterative improvements by adding good features to our model.  The code for this problem is located in the helper functions section below.  Scroll down now and take a look at the code. 

**Creating a Baseline**: When starting to evaluate the usefulness of features, it is usually a good idea to create a baseline.  That is, run your model with little to no features and see how the model performs.  The following code will run logistic regression with only a bias feature. 

In [ ]:
part_of_speech(limit=500)

OK, so now we know that simply making predictions based on the number of occurrences of each type of speech leads to a *training accuracy* of around 52% and a *validation accuracy* of around 56%.  Hopefully we can improve on this by actually giving the model some useful features.  

The obvious choice is to actually tell the model what the words is. 

In [ ]:
part_of_speech(limit=500, word=True)

OK, that seems more reasonable.  Now you should have a training accuracy of around 96% and a validation accuracy of around 79%.  It looks like using the words alone as features induces some overfitting on the training set. Let's see if we can think of new features that we could include that might help.  

Here is where you have to use your brain to do feature engineering!  

The code has output some useful information that you can use to do error analysis and hopefully come up with some useful features.  

The top of the output gives you examples of the features that were used for several examples.  Since we only used the words as features, each example only includes the word itself. 

The next useful piece of information is shown in the *confusion matrix*. Sci-Kit Learn's confusion matrix function returns a matric $C$ such that $C_{ij}$ gives the number of examples known to be in group $i$ that were labeled as group $j$.  From the confusion matrix in the output we see that, in particular, the model is classifying a lot of words that should be verbs as nouns.   

Now, based on this knowledge you might have some ideas about new features you can include to combat this error, but maybe you don't?  It's almost always a good idea to dig into the actual data and look at *specific* examples that your model has misclassified.  To help you with this, the code has printed some of the most common misclassifications.  

**Q**: Looking at the common misclassifications, can you think of a good new feature to add? 

Add your new feature to the model and see how it does! 

In [ ]:
part_of_speech(limit=500, word=True, all_before=True)

**Q**:  Did your model do better on the test data?  Take a look at the confusion matrix and the examples of misclassifications and see if you can think of another new feature to add! 

In [ ]:
part_of_speech(limit=500, )

**Q**:  What's your performance like now?  Repeat this iterative process of adding new features until you're happy with your model (or you've exhausted the number of features available in the code) 

In [ ]:
part_of_speech(limit=500, )

<br><br><br><br>
<br><br><br><br>
<br><br><br><br>
<br><br><br><br>
<br><br><br><br>

<a id='helpers'></a>

<br> 

### Helper Functions
***

In [ ]:
from collections import defaultdict
import string
import operator

import numpy as np

from sklearn.metrics import confusion_matrix
from sklearn.linear_model import SGDClassifier
from sklearn.feature_extraction.text import HashingVectorizer
from sklearn.metrics import accuracy_score

import nltk
nltk.download('wordnet')
nltk.download('brown')

from nltk.corpus import wordnet as wn
from nltk.corpus import brown
from nltk.util import ngrams

import seaborn as sn 
import matplotlib.pyplot as plt
%matplotlib inline

mycolors = {"blue": "steelblue", "red": "#a76c6e", "green": "#6a9373"}


def normalize_tags(tag):
    if not tag or not tag[0] in "ABCDEFGHIJKLMNOPQRSTUVWXYZ":
        return "PUNC"
    else:
        return tag[:2]


kTAGSET = ["", "JJ", "NN", "PP", "RB", "VB"]

class Analyzer:
    def __init__(self, word, before, after, prev, next, char, dict):
        self.word = word
        self.after = after
        self.before = before
        self.prev = prev
        self.next = next
        self.dict = dict
        self.char = char

    def __call__(self, feature_string):
        feats = feature_string.split()

        if self.word:
            yield feats[0]

        if self.after:
            for ii in [x for x in feats if x.startswith("A:")]:
                yield ii
        if self.before:
            for ii in [x for x in feats if x.startswith("B:")]:
                yield ii
        if self.prev:
            for ii in [x for x in feats if x.startswith("P:")]:
                yield ii
        if self.next:
            for ii in [x for x in feats if x.startswith("N:")]:
                yield ii
        if self.dict:
            for ii in [x for x in feats if x.startswith("D:")]:
                yield ii
        if self.char:
            for ii in [x for x in feats if x.startswith("C:")]:
                yield ii
                
def example(sentence, position):
        word = sentence[position][0]
        ex = word
        tag = normalize_tags(sentence[position][1])
        if tag in kTAGSET:
            target = kTAGSET.index(tag)
        else:
            target = None

        if position > 0:
            prev = " P:%s" % sentence[position - 1][0]
        else:
            prev = ""

        if position < len(sentence) - 1:
            next = " N:%s" % sentence[position + 1][0]
        else:
            next = ''

        all_before = " " + " ".join(["B:%s" % x[0] for x in sentence[:position]])
        all_after = " " + " ".join(["A:%s" % x[0] for x in sentence[(position + 1):]])

        dictionary = ["D:ADJ"] * len(wn.synsets(word, wn.ADJ)) + \
          ["D:ADV"] * len(wn.synsets(word, wn.ADV)) + \
          ["D:VERB"] * len(wn.synsets(word, wn.VERB)) + \
          ["D:NOUN"] * len(wn.synsets(word, wn.NOUN))

        dictionary = " " + " ".join(dictionary)

        char = ' '
        padded_word = "~%s^" % sentence[position][0]
        for ngram_length in range(2, 5):
            char += ' ' + " ".join("C:%s" % "".join(cc for cc in x)
                                   for x in ngrams(padded_word, ngram_length))
        ex += char

        ex += prev
        ex += next
        ex += all_after
        ex += all_before
        ex += dictionary

        return ex, target
    
def all_examples(limit, train=True):
    sent_num = 0
    for ii in brown.tagged_sents():
        sent_num += 1
        if limit > 0 and sent_num > limit:
            break

        for jj in range(len(ii)):
            ex, tgt = example(ii, jj)
            if tgt:
                if train and sent_num % 5 != 0:
                    yield ex, tgt
                if not train and sent_num % 5 == 0:
                    yield ex, tgt
                    
def accuracy(classifier, x, y, examples):
    predictions = classifier.predict(x)
    cm = confusion_matrix(y, predictions)

    print("Accuracy: %f" % accuracy_score(y, predictions))

    print("\nConfusion Matrix:\n")
    print("\t".join(kTAGSET[1:]))
    for ii in cm:
        print("\t".join(str(x) for x in ii))

    errors = defaultdict(int)
    for ii, ex_tuple in enumerate(examples):
        ex, tgt = ex_tuple
        if tgt != predictions[ii]:
            errors[(ex.split()[0], kTAGSET[predictions[ii]], kTAGSET[tgt])] += 1

    print("\nSome misclassified examples:\n")
    for ww, cc in sorted(errors.items(), key=operator.itemgetter(1), reverse=True)[:10]:
        print("word: {:10s}  prediced pos: {:2s}      true pos: {:2s}".format(ww[0], ww[1], ww[2]))
        
def part_of_speech(**kwargs):
    word = kwargs.get("word", False)
    all_before = kwargs.get("all_before", False)
    all_after = kwargs.get("all_after", False)
    one_before = kwargs.get("one_before", False)
    one_after = kwargs.get("one_after", False)
    characters = kwargs.get("characters", False)
    dictionary = kwargs.get("dictionary", False)
    limit= kwargs.get("limit",-1)
    
    analyzer = Analyzer(word, all_before, all_after, one_before, one_after, characters, dictionary)
    
    vectorizer = HashingVectorizer(analyzer=analyzer)

    x_train = vectorizer.fit_transform(ex for ex, tgt in all_examples(limit))
    x_valid = vectorizer.fit_transform(ex for ex, tgt in all_examples(limit, train=False))
    
    print("Example Features:")
    exstr = "\n"
    for ex, tgt in all_examples(1):
        exstr += " ".join(analyzer(ex)) + "\n"
    if exstr.replace(" ", "").replace("\n", "") == "":
        print("\n")
    else:
        print(exstr)

    y_train = np.array(list(tgt for ex, tgt in all_examples(limit)))
    y_valid = np.array(list(tgt for ex, tgt in all_examples(limit, train=False)))

    lr = SGDClassifier(loss='log', penalty='l2', tol=None, max_iter=5, shuffle=True)
    lr.fit(x_train, y_train)

    print("\nTraining Set\n-------------------------")
    accuracy(lr, x_train, y_train, all_examples(limit))
    print("\nValidation Set\n--------------------")
    accuracy(lr, x_valid, y_valid, all_examples(limit, train=False))
    
np.random.seed(1234)
    
def income_data(N=200):
    x = 1.1*np.random.normal(size=N)
    y = np.exp(x)
    return y

from IPython.core.display import HTML
HTML("""
<style>
.MathJax nobr>span.math>span{border-left-width:0 !important};
</style>
""")